# Fundamentals of Neural (Week 2)

### MNIST Dataset

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import numpy as np
import tensorflow as tf

#memasukkan url tempat data diambil
DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

#menggunakan function get_file dari keras, untuk mengambil file data dari url yang sudah di set
path = tf.keras.utils.get_file('mnist.npz', DATA_URL)

#memasukkan data ke numpy array, kemudian membaginya ke variable x_train, y_train, x_test, dan y_test
with np.load(path) as data:
  train_examples = data['x_train']
  train_labels = data['y_train']
  test_examples = data['x_test']
  test_labels = data['y_test']

In [3]:
#memasukkan variable ke dataset yang akan digunakan untuk training dan testing
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

#membuat model sequential
model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
#melakukan konfigurasi model, menentukan optimizer, Loss, dan metrics
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])
#train model sebanyak epochs menggunakan dataset
model.fit(train_dataset, epochs=10)
#evaluasi model menggunakan test dataset, output berupa Loss dan accuracy
model.evaluate(test_dataset)

Epoch 1/10
938/938 [==============================] - 3s 3ms/step - loss: 2.7760 - sparse_categorical_accuracy: 0.8704
Epoch 2/10
938/938 [==============================] - 3s 4ms/step - loss: 0.4653 - sparse_categorical_accuracy: 0.9288
Epoch 3/10
938/938 [==============================] - 3s 4ms/step - loss: 0.3552 - sparse_categorical_accuracy: 0.9457
Epoch 4/10
938/938 [==============================] - 3s 4ms/step - loss: 0.2894 - sparse_categorical_accuracy: 0.9528
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2611 - sparse_categorical_accuracy: 0.9593
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2356 - sparse_categorical_accuracy: 0.9632
Epoch 7/10
938/938 [==============================] - 3s 4ms/step - loss: 0.2106 - sparse_categorical_accuracy: 0.9661
Epoch 8/10
938/938 [==============================] - 4s 4ms/step - loss: 0.1999 - sparse_categorical_accuracy: 0.9698
Epoch 9/10
938/938 [============================

[0.5068194270133972, 0.9480000138282776]

### 2. Heart Dataset

In [4]:
import pandas as pd
import tensorflow as tf
#mengambiL data dari url
csv_file_path = '/content/heart.csv'
df = pd.read_csv(csv_file_path)
#Convert thal column dari tipe object menjadi discrete numerical value.
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes
#memasukkan data ke dalam dataset dan melakukan shuffle data untuk training
target = df.pop('target')
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

train_dataset = dataset.shuffle(len(df)).batch(1)

#meLakukan konfigurasi model
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras. layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])
  model.compile(optimizer='adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

  return model
#memasukkan data ke dalam model
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

FileNotFoundError: [Errno 2] No such file or directory: '/content/heart.csv'

### Challenge Corona Dataset

In [ ]:
csv_file_path = '/content/corona.csv'
df = pd.read_csv(csv_file_path)

In [ ]:
#Convert thal column dari tipe object menjadi discrete numerical value.
df['runnyNose'] = pd.Categorical(df['runnyNose'])
df['runnyNose'] = df.runnyNose.cat.codes
#memasukkan data ke dalam dataset dan melakukan shuffle data untuk training
target = df.pop('infProb')
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

train_dataset = dataset.shuffle(len(df)).batch(1)

In [ ]:
import tensorflow as tf

def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    return model

model = get_compiled_model()

In [ ]:
# Convert data to a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
train_dataset = dataset.shuffle(len(df)).batch(1)
# Train the model
model.fit(train_dataset, epochs=15)

## Text Classification
### Basic Text Classification

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

filepath_dict = {'yelp': 'yelp_labelled.txt',
                'amazon': 'amazon_cells_labelled.txt',
                'imdb' : 'imdb_labelled.txt'}
df_list = []

for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['sentence', 'label'], sep='\t')
    df['source'] = source # Add another column filled with the source name
    df_list.append(df)
df = pd.concat(df_list)
print(df.iloc[0])

df_yelp = df[df['source'] == 'yelp']
sentences = df_yelp['sentence'].values
y = df_yelp['label'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(
  sentences, y, test_size=0.25, random_state=1000)


vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test= vectorizer.transform(sentences_test)
classifier = LogisticRegression()
classifier.fit(X_train,y_train)
score = classifier.score(X_test, y_test)
print("Accuracy:", score)

### 5. Text Classification with Neural Network

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import models
from tensorflow.keras import layers

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i-3, '?') for i in train_data[0]])

import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] =1.
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
from tensorflow.keras import losses

model.compile(optimizer='rmsprop',loss=losses.binary_crossentropy,metrics=['acc'])

x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

history = model.fit(partial_x_train,partial_y_train,epochs=20, batch_size=512, validation_data=(x_val,y_val))
history_dict = history.history
history_dict.keys()

import matplotlib.pyplot as plt

epochs = range(1, len(history_dict['acc']) + 1)
plt.plot(epochs, history_dict['loss'], 'bo', label='Training loss')
plt.plot(epochs, history_dict['val_loss'], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel(' Epochs')
plt.ylabel(' Loss ')
plt.legend()
plt.show()

In [ ]:
#Plotting the training and validation accuracy
plt.clf() #clears the figures
acc_values = history_dict['val_acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, history_dict['acc'], 'bo', label='Training Acc')
plt.plot(epochs, history_dict['val_acc'], 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Menggunakan neural network (no 5) membuat kerja algoritma lebih cerdas untuk mengklasifikasikan teks tersebut dibandingkan cara basic. Hasil yang didapatkan neural network sebesar loss: 0.0198 - acc: 0.9965 - val_loss: 0.5619 - val_acc: 0.872, dibandingkan basic yang nilai loss: 0.7257 - accuracy: 0.5082